# Fire detection pipeline

First malfunctioning pixel mask gets created by using the dead and drifting pixel mask functions, which later gets applied on the data.

Then clusters of fire pixels in each frame are identified. For L2 and M0 bands, a fixed threshold is used, while for L1 an adaptive threshold is applied. Clusters are dilated and border contrast is checked to reduce false positives.

Detected clusters are tracked across frames based on centroid movement, speed, and direction. Only tracks meeting minimum criteria (speed, length, spatial consistency) are retained.

Centroids from L2 and L1 tracks are paired if they appear in corresponding frames (with a defined offset) and are spatially close. Only L2 clusters with at least one paired L1 centroid are considered verified.

Verified L2 clusters are saved as a GeoTIFF mask in `out_path`.
Detailed information about each verified cluster (track ID, frame, coordinates) is stored in `fire_data`.

functions for malfunctioning pixels

In [1]:
import os
import glob
import numpy as np
import xarray as xr
import rasterio
from scipy.ndimage import label, binary_dilation, median_filter
import rasterio
from collections import Counter

# load npy files and extract bands into xarray dataset

def load_frames_to_xarray(npy_folder):
    def extract_number(filename):
        # Extract the number from the filename, e.g., "frame_12.npy" -> 12
        basename = os.path.basename(filename)
        num = ''.join(filter(str.isdigit, basename))
        return int(num) if num else -1

    npy_files = sorted(glob.glob(os.path.join(npy_folder, "*.npy")), key=extract_number)
    # print("Read order:", npy_files)

    L2_list = []
    L1_list = []
    M0_list = []

    for f in npy_files:
        arr = np.load(f)
        L2 = arr[0:100, :, 0]      # L2: rows 0-100, band 0
        L1 = arr[349:419, :, 1]    # L1: rows 349-419, band 1
        M0 = arr[668:768, :, 2]    # M0: rows 668-767, band 2
        L2_list.append(L2)
        L1_list.append(L1)
        M0_list.append(M0)

    L2_arr = np.stack(L2_list)  # shape: (frame, x, y)
    L1_arr = np.stack(L1_list)
    M0_arr = np.stack(M0_list)

    ds = xr.Dataset(
        {
            "L2": (["frame", "x_L2", "y_L2"], L2_arr),
            "L1": (["frame", "x_L1", "y_L1"], L1_arr),
            "M0": (["frame", "x_M0", "y_M0"], M0_arr)
        },
        coords={
            "frame": np.arange(L2_arr.shape[0]),
            "x_L2": np.arange(L2_arr.shape[1]),
            "y_L2": np.arange(L2_arr.shape[2]),
            "x_L1": np.arange(L1_arr.shape[1]),
            "y_L1": np.arange(L1_arr.shape[2]),
            "x_M0": np.arange(M0_arr.shape[1]),
            "y_M0": np.arange(M0_arr.shape[2])
        }
    )
    return ds

# create drifting pixel mask based on temporal std and local median deviation
# mode: "L2", "L1" or "M0"
# save as tiff if wanted
def create_dead_pixel_mask(ds, mode, out_path, transform = None):
    if mode == "L2":
        band_name = "L2"
    elif mode == "L1":
        band_name = "L1"
    elif mode == "M0":
        band_name = "M0"
    else:
        raise ValueError("mode must be 'L2', 'L1', or 'M0'")

    band = ds[band_name].values

    # Initialize mask with NaNs from the first frame
    current_nan_mask = np.isnan(band[0]).copy()

    # Track added and removed NaNs across frames, update mask accordingly
    for frame_idx in range(1, band.shape[0]):
        frame_nan_mask = np.isnan(band[frame_idx])
        current_nan_mask |= frame_nan_mask

    # Save the final (updated) NaN mask as GeoTIFF
    dead_mask_final = current_nan_mask.astype("float32") * 10000
    # band_out_path = os.path.join(out_path, f"dead_pixel_mask_{band_name}.tif")
    # with rasterio.open(
    #     band_out_path,
    #     "w",
    #     driver="GTiff",
    #     height=dead_mask_final.shape[0],
    #     width=dead_mask_final.shape[1],
    #     count=1,
    #     dtype="float32",
    #     transform=transform,
    #     crs="EPSG:4326"
    # ) as dst:
    #     dst.write(dead_mask_final, 1)

    # print(f"Dead pixel mask (all frames) saved at: {band_out_path}")
    return dead_mask_final

def create_drifting_pixel_mask(ds, mode, out_path, transform = None):
    if mode == "L2":
        band_name = "L2"
        x_dim = "x_L2"
        y_dim = "y_L2"
    elif mode == "L1":
        band_name = "L1"
        x_dim = "x_L1"
        y_dim = "y_L1"
    elif mode == "M0":
        band_name = "M0"
        x_dim = "x_M0"
        y_dim = "y_M0"
    else:
        raise ValueError("Mode must be 'L2', 'L1' or 'M0'.")

    band = ds[band_name].values
    median_band = np.nanmedian(band, axis=0)
    local_median = median_filter(median_band, size=5)
    local_dev = np.abs(median_band - local_median)
    mad = np.nanmean(local_dev)
    outlier_mask_local = local_dev > 8 * mad
    temporal_std = np.nanstd(band, axis=0)
    const_mask = temporal_std < 1

    # Drifting: constant or local outlier
    drifting_mask = (const_mask) | (outlier_mask_local)
    drifting_mask_float = drifting_mask.astype("float32") * 10000
    ds[f"drifting_pixel_mask_{band_name}"] = ([x_dim, y_dim], drifting_mask_float)

    # drifting_out_path = os.path.join(out_path, f"drifting_pixel_mask_{band_name}.tif")
    # with rasterio.open(
    #     drifting_out_path,
    #     "w",
    #     driver="GTiff",
    #     height=drifting_mask_float.shape[0],
    #     width=drifting_mask_float.shape[1],
    #     count=1,
    #     dtype="float32",
    #     transform=transform,
    #     crs="EPSG:4326"
    # ) as dst:
    #     dst.write(drifting_mask_float, 1)
    # print(f"Drifting pixel mask saved at: {drifting_out_path}")
    return drifting_mask_float

# combine dead and drifting pixel masks
# save as tiff if wanted

def create_malfunctioning_pixel_mask(dead_mask, drifting_mask, mode, out_path, transform = None):

    if mode == "L2":
        band_name = "L2"
        x_dim = "x_L2"
        y_dim = "y_L2"
    elif mode == "L1":
        band_name = "L1"
        x_dim = "x_L1"
        y_dim = "y_L1"
    elif mode == "M0":
        band_name = "M0"
        x_dim = "x_M0"
        y_dim = "y_M0"
    else:
        raise ValueError("Mode must be 'L2', 'L1' or 'M0'.")

    malfunctioning_mask = dead_mask.astype(bool) | drifting_mask.astype(bool)
    malfunctioning_mask_float = malfunctioning_mask.astype("float32") * 10000

    # out_path = os.path.join(out_path, f"malfunctioning_pixel_mask_{band_name}.tif")
    # with rasterio.open(
    #     out_path,
    #     "w",
    #     driver="GTiff",
    #     height=malfunctioning_mask_float.shape[0],
    #     width=malfunctioning_mask_float.shape[1],
    #     count=1,
    #     dtype="float32",
    #     transform=transform,
    #     crs="EPSG:4326"
    # ) as dst:
    #     dst.write(malfunctioning_mask_float, 1)
    # print(f"Malfunctioning pixel mask saved at: {out_path}")
    return malfunctioning_mask_float

functions for fire detection and verification

In [ ]:
# checks each frame for clusters of fire pixels above a threshold, dilates them and checks border contrast
# saves masks as tiff if wanted
# output:
# all_masks is a list of 2D numpy arrays (one per frame).
# Each array is a binary mask: 1 marks pixels belonging to a detected fire cluster (including border), 0 means background.
# The length of the list equals the number of frames; each mask has the same shape
def save_fire_cluster_masks(valid, mal_mask, output_folder, transform, 
                                      target_value=None, border_contrast=30, adaptive=False):
        os.makedirs(output_folder, exist_ok=True)
        valid = np.where(mal_mask, np.nan, valid)
        all_masks = []
        for frame_idx in range(valid.shape[0]):
            frame = valid[frame_idx]

            if adaptive:
                # adaptive threshold because fire is not always that much brighter than background in L1
                frame_mean = np.nanmean(frame)
                frame_std = np.nanstd(frame)
                threshold = frame_mean + 2.5 * frame_std

                local_med = median_filter(
                    np.nan_to_num(frame, nan=frame_mean), size=7, mode='nearest'
                )
                contrast = frame - local_med
                mask = (frame > threshold) & (contrast > border_contrast)

            else:
                # fixed threshold
                mask = (frame > target_value)

            labeled, ncomp = label(mask)
            combined_mask = np.zeros_like(frame, dtype=bool)

            for cluster_id in range(1, ncomp + 1):
                cluster_mask = (labeled == cluster_id)
                dilated = binary_dilation(cluster_mask, iterations=1)
                border_mask = dilated & ~cluster_mask

                # Border-Kontrastcheck
                non_fire = ~mask
                frame_non_fire = np.where(non_fire, frame, np.nan)
                local_mean = median_filter(
                    np.nan_to_num(frame_non_fire, nan=np.nanmean(frame_non_fire)), size=10, mode='nearest'
                )
                border_diff = np.abs(frame - local_mean)
                border_mask = border_mask & (border_diff > border_contrast)

                cluster_plus_border = cluster_mask | border_mask
                combined_mask |= cluster_plus_border

            all_masks.append(combined_mask.astype(np.uint8)) 
            mask_uint8 = combined_mask.astype(np.uint8)

            # out_path = os.path.join(output_folder, f"{mode}_frame_{frame_idx}_all_clusters_mask.tif")
            # with rasterio.open(
            #     out_path,
            #     "w",
            #     driver="GTiff",
            #     height=mask_uint8.shape[0],
            #     width=mask_uint8.shape[1],
            #     count=1,
            #     dtype="uint8",
            #     transform=transform,
            #     crs="EPSG:4326"
            # ) as dst:
            #     dst.write(mask_uint8, 1)
        return all_masks




# function to verify fire tracks based on speed, direction and length, it uses centroids of clusters
# Output:
# filtered_tracks is a list of fire track dictionaries.
# Each dictionary contains:
# 'frames': list of frame indices where the track appears,
# 'centroids': list of (row, col) coordinates for the cluster centroid in each frame.
# Only tracks that meet minimum criteria (e.g. length, speed) are included.
def track_fire_mask_clusters(mask_stack, min_speed=1, max_speed=30, max_col_shift=10, min_track_length=2):
        firetracks = []
        active = []
        for t, mask in enumerate(mask_stack):
            labeled, ncomp = label(mask)
            centroids = []
            sizes = []
            for lab in range(1, ncomp+1):
                coords = np.argwhere(labeled == lab)
                if coords.size == 0:
                    continue
                r = coords[:,0].mean()
                c = coords[:,1].mean()
                centroids.append((r, c))
                sizes.append(coords.shape[0])
            new_active = []
            used = set()
            for tr in active:
                last_r, last_c = tr["centroids"][-1]
                best = None
                best_dist = 1e9
                for i, (r, c) in enumerate(centroids):
                    if i in used:
                        continue
                    dr = r - last_r
                    dc = c - last_c
                    speed = abs(dr)
                    if min_speed <= speed <= max_speed and abs(dc) <= max_col_shift:
                        dist = abs(dr) + abs(dc)
                        if dist < best_dist:
                            best_dist = dist
                            best = (i, r, c)
                if best:
                    i, r, c = best
                    used.add(i)
                    tr["frames"].append(t)
                    tr["centroids"].append((r, c))
                    new_active.append(tr)
                else:
                    firetracks.append(tr)
            for i, (r, c) in enumerate(centroids):
                if i not in used:
                    new_active.append({"frames": [t], "centroids": [(r, c)]})
            active = new_active
        firetracks.extend(active)
        filtered_tracks = [tr for tr in firetracks if len(tr["frames"]) >= min_track_length]
        return filtered_tracks



# saves all firetrack masks into one joint mask
def save_joint_firetrack_mask(firetracks, all_masks, out_path, transform, filename=None):
        joint_mask = np.zeros((len(all_masks), all_masks[0].shape[0], all_masks[0].shape[1]), dtype=np.uint8)
        for tr in firetracks:
            for t, (r, c) in zip(tr["frames"], tr["centroids"]):
                rr = int(round(r))
                cc = int(round(c))
                if 0 <= rr < joint_mask.shape[1] and 0 <= cc < joint_mask.shape[2]:
                    joint_mask[t, rr, cc] = 1

        joint_mask_sum = np.any(joint_mask, axis=0).astype(np.uint8)
        # if filename is None:
        #     filename = f"{mode}_joint_tracks_mask_sum.tif"
        # out_path = os.path.join(out_path, filename)
        # with rasterio.open(
        #     out_path,
        #     "w",
        #     driver="GTiff",
        #     height=joint_mask_sum.shape[0],
        #     width=joint_mask_sum.shape[1],
        #     count=1,
        #     dtype="uint8",
        #     transform=transform,
        #     crs="EPSG:4326"
        # ) as dst:
        #     dst.write(joint_mask_sum, 1)
        # print(f"combined track mask: {out_path}")




# combines all steps above for one mode
# output:
# all_masks is a list of 2D numpy arrays (one per frame), each representing the fire mask for that frame.
# firetracks is a list of dictionaries, each describing a detected fire track with its frame indices and centroids.
# saves firetrack masks as tiff if wanted
def process_fire_detection(mode, ds, out_path, transform, malfunctioning_mask):
    if mode == "L2":
        valid = ds["L2"].values
        mal_mask = malfunctioning_mask 
        target_value = -650
        border_contrast = 40
        adaptive = False
    elif mode == "L1":
        valid = ds["L1"].values
        mal_mask = malfunctioning_mask 
        target_value = None
        border_contrast = 20
        adaptive = True
    elif mode == "M0":
        valid = ds["M0"].values
        mal_mask = malfunctioning_mask 
        target_value = 300
        border_contrast = 40
        adaptive = False
    else:
        raise ValueError("Mode must be 'L2', 'L1' or 'M0'.")

    all_masks = save_fire_cluster_masks(valid, mal_mask, out_path, transform, 
                                       target_value=target_value, border_contrast=border_contrast, adaptive=adaptive)
    firetracks = track_fire_mask_clusters(all_masks)
    print(f"Detected {len(firetracks)} fire tracks in {mode} data.")
    for idx, tr in enumerate(firetracks):
        print(f"Track {idx+1}: Frames {tr['frames']}, Centroids {tr['centroids']}")

    #save_joint_firetrack_mask(firetracks, all_masks, out_path, transform)
    return all_masks, firetracks




# collects centroids from tracks into a list of dicts
# Each dictionary contains:
# 'band': string indicating the band ("L1" or "L2"),
# 'track_id': integer (ID of the fire track),
# 'frame': integer (frame number),
# 'row': float (row coordinate of the centroid),
# 'col': float (column coordinate of the centroid)
def collect_centroids(tracks, band):
    centroids = []
    for track_id, tr in enumerate(tracks):
        for frame, (row, col) in zip(tr['frames'], tr['centroids']):
            centroids.append({
                'band': band,
                'track_id': track_id,
                'frame': frame,
                'row': row,
                'col': col
            })
    return centroids



# checks if L1 centroids match L2 centroids based on frame offset and spatial tolerance
# output: list of paired centroids as tuples (L2_centroid_dict, L1_centroid_dict)
def pair_centroids(centroids_L2, centroids_L1, frame_offset=28, row_tol=20, col_tol=20):
    pairs = []
    for c2 in centroids_L2:
        for c1 in centroids_L1:
            if abs(c2['frame'] - c1['frame']) == frame_offset:
                if abs(c2['row'] - c1['row']) <= row_tol and abs(c2['col'] - c1['col']) <= col_tol:
                    pairs.append((c2, c1))
    return pairs




# creates a joint mask of all L2 clusters that have a paired L1 centroid
# joint_L2_cluster_mask is a 2D numpy array (uint8).
# Pixels belonging to verified clusters are set to 1, all others are 0.
def create_joint_L2_cluster_mask(centroids_L2, paired_track_ids, all_masks, max_dist=5):
    joint_L2_cluster_mask = np.zeros_like(all_masks[0], dtype=np.uint8)
    for centroid in centroids_L2:
        track_id = centroid['track_id']
        if track_id in paired_track_ids:
            frame_idx = centroid['frame']
            if 0 <= frame_idx < len(all_masks):
                row = int(round(centroid['row']))
                col = int(round(centroid['col']))
                labeled, ncomp = label(all_masks[frame_idx])
                cluster_label = labeled[row, col]
                if cluster_label > 0:
                    cluster_mask = (labeled == cluster_label)
                    coords = np.argwhere(cluster_mask)
                    for r, c in coords:
                        dist = np.sqrt((r - row) ** 2 + (c - col) ** 2)
                        if dist <= max_dist:
                            joint_L2_cluster_mask[r, c] = 1
    return joint_L2_cluster_mask


function for pipeline

In [ ]:
# creates masks for L1 and L2 and verifies L2 tracks based on L1 tracks
# saves verified L2 clusters as tiff
# saves information about all verified L2 clusters, including fireid, frames where they appear and their coordinates
def run_fire_detection_pipeline(npy_folder, out_path):
    ds = load_frames_to_xarray(npy_folder)

    # L1 detection
    mode = "L1"
    dead_mask_L1 = create_dead_pixel_mask(ds, mode, out_path, transform=None)
    drifting_mask_L1 = create_drifting_pixel_mask(ds, mode, out_path, transform=None)
    malfunctioning_mask_L1 = create_malfunctioning_pixel_mask(dead_mask_L1, drifting_mask_L1, mode, out_path, transform=None)
    all_fire_masks_L1, fire_tracks_L1 = process_fire_detection(mode, ds, out_path, transform=None,malfunctioning_mask=malfunctioning_mask_L1)

    # L2 detection
    mode = "L2"
    dead_mask_L2 = create_dead_pixel_mask(ds, mode, out_path, transform=None)
    drifting_mask_L2 = create_drifting_pixel_mask(ds, mode, out_path, transform=None)
    malfunctioning_mask_L2 = create_malfunctioning_pixel_mask(dead_mask_L2, drifting_mask_L2, mode, out_path, transform=None)
    all_fire_masks_L2, fire_tracks_L2 = process_fire_detection(mode, ds, out_path, transform=None, malfunctioning_mask=malfunctioning_mask_L2)

    # Track clusters and collect centroids
    centroids_L2 = collect_centroids(fire_tracks_L2, 'L2')
    centroids_L1 = collect_centroids(fire_tracks_L1, 'L1')

    # Pair centroids
    paired_centroids = pair_centroids(centroids_L2, centroids_L1, frame_offset=28, row_tol=20, col_tol=20)
    print(f"found pairs: {len(paired_centroids)}")
    paired_track_ids = set([pair[0]['track_id'] for pair in paired_centroids])

    # Joint mask for verified L2 tracks
    joint_L2_cluster_mask = create_joint_L2_cluster_mask(centroids_L2, paired_track_ids, all_fire_masks_L2, max_dist=5)
    out_path_L2_clusters = os.path.join(out_path, "fire_tracks_L2_clusters_verified.tif")
    with rasterio.open(
        out_path_L2_clusters,
        "w",
        driver="GTiff",
        height=joint_L2_cluster_mask.shape[0],
        width=joint_L2_cluster_mask.shape[1],
        count=1,
        dtype="uint8",
        transform=None,
        crs="EPSG:4326"
    ) as dst:
        dst.write(joint_L2_cluster_mask, 1)
    print(f"fire tracks in L2 with min 1 paired in L1 saved at: {out_path_L2_clusters}")


    track_data = []
    # Overview of detected fire tracks in L2 with at least one paired track in L1
    print("\nFire tracks in L2 with at least one paired track in L1 (cluster coordinates):")
    for track_id in paired_track_ids:
        track = fire_tracks_L2[track_id]
        print(f"Track ID: {track_id}")
        for frame_idx, centroid in zip(track['frames'], track['centroids']):
            mask = all_fire_masks_L2[frame_idx]
            labeled, ncomp = label(mask)
            row = int(round(centroid[0]))
            col = int(round(centroid[1]))
            cluster_label = labeled[row, col]
            if cluster_label > 0:
                cluster_coords = np.argwhere(labeled == cluster_label)
                track_data.append({
                    'track_id': track_id,
                    'frame': frame_idx,
                    'cluster_coords': cluster_coords.tolist()
                })
                print(f"  Frame: {frame_idx}, Cluster size: {len(cluster_coords)}")
                print(f"  Cluster coordinates (first 10): {cluster_coords[:10]}")
        print("-" * 40)
    return track_data

# execution

In [ ]:
npy_folder = "/Users/simonreichl/Desktop/Cal_Val working student challenge/frames"  # Replace with your npy files folder
out_path = "/Users/simonreichl/Desktop/Cal_Val working student challenge/output"  # Replace with your desired output folder

# Run the pipeline
fire_data = run_fire_detection_pipeline(npy_folder, out_path)

# Each entry in fire_data is a dictionary with:
# 'track_id': integer (ID of the fire track)
# 'frame': integer (frame number)
# 'cluster_coords': list of [row, col] pairs, representing the pixel coordinates of the cluster